In [1]:
import sys
import os
sys.path.append(os.path.abspath("../.."))
import numpy as np
import yfinance as yf
import plotly.express as px
from tinyshift.series import relative_strength_index, bollinger_bands, adi_cv, foreca, maximum_achievable_accuracy, sample_entropy, hampel_filter, hurst_exponent, standardize_returns
from tinyshift.stats import trailing_window
from numpy.random import standard_normal

In [2]:
ticker = 'PETR4.SA'
period = '2y'
data = yf.download(ticker, period=period, interval="1d", auto_adjust=True)
t = np.linspace(0, 10, 1000)
pure_sine = np.sin(2 * np.pi * 1 * t)  # Perfectly forecastable

[*********************100%***********************]  1 of 1 completed


In [3]:
r = standardize_returns(data["Close"]["PETR4.SA"])

# RSI

In [4]:
vol_rsi = relative_strength_index(data["Close"]["PETR4.SA"], rolling_window=14)

In [5]:
px.line(vol_rsi, title=f"{ticker} Closing Prices").show(fig_type="png")

# Entropy Volatility

In [6]:
vol_ent = trailing_window(r, rolling_window=60, func=sample_entropy, m=1)

In [7]:
px.line(vol_ent, title=f"{ticker} Closing Prices").show(fig_type="png")

In [8]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    y=data["Close"]["PETR4.SA"],
    x=data.index,
    name="Preço PETR4.SA",
    yaxis="y1"
))

fig.add_trace(go.Scatter(
    x=data.index,
    y=vol_ent,
    name="Volatility Entropy",
    yaxis="y2"
))

fig.add_trace(go.Scatter(
    x=data.index,
    y=vol_rsi,
    name="Volatility RSI",
    yaxis="y3"
))


fig.update_layout(
    yaxis3=dict(
        title="Volatility RSI",
        anchor="free",
        overlaying="y",
        side="right",
        position=1.0,
        showgrid=False
    )
)

fig.update_layout(
    title="Preço PETR4.SA e Volatility Entropy",
    xaxis_title="Data",
    yaxis=dict(title="Preço PETR4.SA"),
    yaxis2=dict(title="Volatility Entropy", overlaying="y", side="right"),
    legend=dict(x=0.01, y=0.99)
)

fig.show(fig_type="png")

# Bollinger's Band

In [9]:
bb = bollinger_bands(data["Close"]["PETR4.SA"])

In [10]:
data["Close"]["PETR4.SA"][bb]

Date
2024-01-03    29.450443
2024-01-25    29.692333
2024-01-26    30.206362
2024-01-29    30.667467
2024-01-30    30.478497
2024-01-31    30.811092
2024-02-01    31.423386
2024-02-29    30.342426
2024-03-08    27.742077
2024-03-11    26.948368
2024-04-10    29.926676
2024-04-22    31.370472
2024-04-25    31.914734
2024-04-26    32.365456
2024-04-29    32.753994
2024-05-17    29.764376
2024-06-14    28.935345
2024-07-01    32.231037
2024-07-29    30.770926
2024-07-30    30.579020
2024-08-02    29.811415
2024-08-05    29.786383
2024-08-26    33.954937
2024-08-28    33.980675
2024-10-07    32.925217
2024-11-14    31.981310
2024-11-18    32.779343
2024-11-19    32.436104
2024-11-22    33.826218
2024-11-25    33.620277
2025-01-31    34.912468
2025-02-12    33.606377
2025-02-18    35.533092
2025-02-19    35.607197
2025-03-05    32.068710
2025-03-06    31.735237
2025-04-07    30.734831
2025-04-08    29.641788
2025-04-10    28.928532
2025-05-12    30.042606
2025-05-13    30.498228
2025-05-14 

# Hampel Filter

In [11]:
hf = hampel_filter(data["Close"]["PETR4.SA"], rolling_window=7)

In [12]:
data["Close"]["PETR4.SA"][hf]

Date
2024-05-22    30.161884
2024-07-04    31.138033
2024-10-09    31.938400
2025-02-07    33.884270
Name: PETR4.SA, dtype: float64

# ADI-CV Framework

In [13]:
adi_cv(data["Volume"]["PETR4.SA"])

(np.float64(1.0), np.float64(0.3049618792144121))

# Sampen

In [14]:
sample_entropy(r, m=3)

np.float64(1.630118117671639)

In [15]:
sample_entropy(pure_sine, m=2)

np.float64(0.16791509477225003)

# Foreca

In [16]:
foreca(data["Close"]["PETR4.SA"])

0.5865781783424653

In [17]:
foreca(pure_sine)

0.9993976808641186

# Maximum Achievable Accuracy

In [18]:
maximum_achievable_accuracy(r, m=1)

np.float64(0.7776350338132325)

In [19]:
maximum_achievable_accuracy(pure_sine, m=1)

np.float64(0.9855199246783082)

# Hurst's Exponent

In [20]:
hurst_exponent(r)

(0.4316761914926692, 0.15713104392666477)

In [21]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    y=data["Close"]["PETR4.SA"],
    x=data.index,
    name="Preço PETR4.SA",
    yaxis="y1"
))


fig.show(fig_type="png")

In [22]:
np.random.seed(42)
n_points = 1000
trend = np.linspace(0, 10, n_points)
noise_trend = np.cumsum(standard_normal(n_points) * 0.1)
mean_reversion = np.zeros(n_points)
for t in range(1, n_points):
    mean_reversion[t] = mean_reversion[t-1] * 0.6 + standard_normal() * 0.5
brownian = np.cumsum(standard_normal(n_points))

In [23]:
hurst_exponent(noise_trend)

(0.6419241912818302, 0.000952023886777138)

In [24]:
hurst_exponent(mean_reversion)

(0.3685698852341455, 0.05402692976986088)

In [25]:
hurst_exponent(pure_sine)

(0.8575205638768405, 0.0014654668633357023)